#### Problem 1
#### a)
Show that:   $ cond_{||.||}   (\alpha A) = cond_{||.||}$

$ cond_{||.||}   (\alpha A) =||\alpha A||~||(\alpha A)^{-1}|| = ||\alpha A||~||(\alpha)^{-1} (A)^{-1} ||= |\alpha| ||A|| |\alpha ^{-1}| ||A^{-1}|| = ||A||~||A^{-1}|| = cond_{||.||}$

#### b)
Show that: $cond_{||.||} (AB) \leq cond_{||.||}(A) cond_{||.||}(B)$ for an invertible $B \in \mathbb{R}^{dxd}$

$cond_{||.||}(AB) = ||AB||~||(AB)^{-1}|| = ||AB||~||A^{-1}B^{-1}|| \leq ||A||~||B||~||A^{-1}||~||B^{-1}|| = ||AA^{-1}||~||BB^{-1}||=cond_{||.||}(A) \cdot cond_{||.||}(B) $

The inequality is from Theorem 1.3

#### c)
Show that: $cond_{||.||}(A) \geq \rho(A) \cdot \rho(A^{-1}) $

From Theorem 1.4 we have that $\rho (A) \leq ||A|| ~~ \forall A \in \mathbb{C}^{2x2} $, where $||.||$ is a consistent matrix.

We then have: 


$cond_{||.||} = ||A||~||A^{-1}|| \geq \rho(A) \rho(A^{-1})$

#### d)
Show that: $cond_{||.||_{2}} (Q) = 1$  for an orhtogonal matrix Q.

For an orthogonal matrix we have $Q^{-1} = Q^{T}$

$cond_{||.||_{2}} (Q) = ||Q||_{2} ||Q^{-1}||_{2} = ||Q||_{2} ||Q^{T}||_{2} = \sqrt{\rho(Q^{T}Q)} \cdot \sqrt{\rho(QQ^{T})} = \sqrt{\rho(I)} \cdot \sqrt{\rho(I)} = 1 \cdot 1 = 1$

#### Problem 4
Compute the Cholesky decomposition of the followin symmetric positive define matrix

$A = \left[ \begin{matrix} 3 & 9 & 6 
\\ 9 & 29 & 16 
\\ 6 & 16 & 17 \end{matrix} \right]$



From Theorem 3.6: Let $A \in \mathbb{R}^{nxn}$ be a symmetric and positive definite matrix. Then there exists a unique upper triangulare matrix H with positive diagonal entries such that

 $A = H H^{T}$ 

 The entries $h_{ij}$ of $H^{T}$ can be computed as follows:

 $h_{11} = \sqrt{a_{11}}$ and, for $ i = 2, ..., n$

 $h_{ij} = (a_{ij} - \sum_{k=1}^{j-1} h_{ik} h_{jk})/h_{jj}$, $j = 1, ..., i-1$

 $h_{ii} =(a_{ii} - \sum_{k=1}^{i-1} h_{ik}^{2})^{1/2}$


 We can then calculate the following coefficients for them matrices $H$ and $H^{T}$:

$h_{11} = \sqrt{3}$

$h_{21} = \dfrac{a_{21}}{h_{11}} = \dfrac{9}{\sqrt{3}} = 3 \sqrt{3}$

$h_{22} = \sqrt{a_{22} - h_{21}^{2}} = \sqrt{29-27} = \sqrt{2}$

$h_{31} = \dfrac{a_{31}}{h_{11}} = \dfrac{6}{\sqrt{3}} = 2 \sqrt{3}$

$h_{32} = \dfrac{a_{32} - (h_{31} h_{21})} {h_{22}} = \dfrac{16 - (2 {\sqrt{3} 3 \sqrt{3}})}{\sqrt{2}} = -\sqrt{2}$

$h_{33} = \sqrt{a_{33} - (h_{31}^{2} + h_{32}^{2})} = \sqrt{17 - ((2\sqrt{3})^{2} + (-\sqrt{2})^{2})} = \sqrt{3}$

We get the matrices:

$H^{T} = \left[ \begin{matrix} 
\sqrt{3} & 3 \sqrt{3} & 2 \sqrt{3} 
\\ 0 & \sqrt{2} & -\sqrt{2}
\\ 0 & 0 & \sqrt{3} \end{matrix} \right]$

$H = \left[ \begin{matrix} 
\sqrt{3} & 0  & 0
\\ 3 \sqrt{3} & \sqrt{2} & 0
\\ 2 \sqrt{3} & -\sqrt{2} & \sqrt{3} \end{matrix} \right]$

#### Problem 2

In [ ]:
import numpy as np

In [ ]:
def mylu(A):
    n = A.shape[0]
    P = np.arange(n)

    for k in range(n-1):
        col = np.abs(A[k:, k]) #extracting column k from the diagonal and down
        P_l = np.argmax(col) + k #index of the maximum value of the column

        P[[k,P_l]] = P[[P_l, k]] #swapping P_k by P_l

        A[[k,P_l]] = A[[P_l, k]] #swapping the rows in A accordingly

        #Perform elimination
        for i in range(k+1, n):
            multiplier = A[i,k] / A[k,k] #find multipliers
            A[i,k:] -= multiplier * A[k,k:]

    return A,P

In [ ]:
A = np.array([[2,5,8,7], [5,2,2,8], [7,5,6,6], [5,4,4,8]])

print(mylu(A))